# Configuration of Deployment

In [7]:
from azureml.core.model import Model, InferenceConfig
from azureml.core import Workspace, Model, Environment
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.webservice import AciWebservice

In [3]:

tenant_id = '178121b9-4c42-4e6f-8611-0350f0190053'
workspace_name = 'mlw-nairobitalk-a001'
subscription_id = '23529470-ba17-4d8a-9f0c-064e63a49c33'
resource_group = 'rg-nairobitalk-a001'
environment_name = 'env-pytorch'
training_compute_name = 'gpu-cluster'
endpoint_name = 'ci-nairobitalk-a001'


## Connect to workspace

In [4]:
credentials = InteractiveLoginAuthentication(tenant_id = tenant_id)
ws = Workspace(subscription_id=subscription_id,
                resource_group=resource_group,
                workspace_name=workspace_name,
                auth = credentials)

## Get Registered Model

In [5]:
model = Model(ws, name='SimpsonsClassification-pytorch')
print("Loaded model version:",model.version)

Loaded model version: 1


## Configure Inference Environment

In [17]:
myenv = Environment(name="simpsons-inference")
myenv.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1'
conda_dep = CondaDependencies(conda_dependencies_file_path='inference/env.yml')
myenv.python.conda_dependencies=conda_dep

In [18]:
inference_config = InferenceConfig(
    entry_script="inference/score.py", 
    environment=myenv
)

deploy_config = AciWebservice.deploy_configuration(
                    cpu_cores = 1, 
                    memory_gb = 2,
                    description='Simpson Lego Classifier')

## Deploy Model

In [19]:
# Deploy the model to an ACI
aci_service = Model.deploy(ws, 
                name="simpsons-pt-aci", 
                models = [model], 
                inference_config = inference_config, 
                deployment_config = deploy_config, 
                overwrite = True)

aci_service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-07-08 20:05:30+03:00 Creating Container Registry if not exists.
2022-07-08 20:05:30+03:00 Registering the environment.
2022-07-08 20:05:32+03:00 Building image.